In [1]:
import json
from django.utils.dateparse import parse_datetime
import os
import sys
import django

# Add the project directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

# Set the Django settings module
os.environ['DJANGO_SETTINGS_MODULE'] = 'wearmai.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Initialize Django
django.setup()

In [2]:
from core.models import (
    Pelvis, PelvisLeftSide, PelvisRightSide,
    Ankle, AnkleLeftSide, AnkleRightSide,
    Knee, KneeLeftSide, KneeRightSide,
    Hip, HipLeftSide, HipRightSide,
    Soleus, SoleusLeftSide, SoleusRightSide,
    TibialisAnterior, TibialisAnteriorLeftSide, TibialisAnteriorRightSide,
    MedialGastrocnemius, MedialGastrocnemiusLeftSide, MedialGastrocnemiusRightSide,
    LateralGastrocnemius, LateralGastrocnemiusLeftSide, LateralGastrocnemiusRightSide,
    GaitPhase, ExerciseUnit, 
    Run, Walk, Jump, Squat, Land, Lunge, 
    UserProfile
)


In [3]:
import json
import pandas as pd
from datetime import datetime
from django.db import transaction
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define your model imports here
# from your_app.models import (
#     Run, Walk, Jump, Squat, Land, Lunge, ExerciseUnit, GaitPhase,
#     Pelvis, Ankle, Knee, Hip, Soleus, MedialGastrocnemius,
#     LateralGastrocnemius, TibialisAnterior,
#     PelvisLeftSide, PelvisRightSide,
#     AnkleLeftSide, AnkleRightSide,
#     KneeLeftSide, KneeRightSide,
#     HipLeftSide, HipRightSide,
#     SoleusLeftSide, SoleusRightSide,
#     MedialGastrocnemiusLeftSide, MedialGastrocnemiusRightSide,
#     LateralGastrocnemiusLeftSide, LateralGastrocnemiusRightSide,
#     TibialisAnteriorLeftSide, TibialisAnteriorRightSide
# )

# Mapping of variable names to their respective categories
VAR_DICT = {
    'pelvis_tilt': 'pelvis',
    'pelvis_list': 'pelvis',
    'pelvis_rotation': 'pelvis',
    'hip_flexion_r': 'hip',
    'hip_flexion_l': 'hip',
    'hip_adduction_r': 'hip',
    'hip_adduction_l': 'hip',
    'hip_rotation_r': 'hip',
    'hip_rotation_l': 'hip',
    'ankle_angle_r': 'ankle',
    'ankle_angle_l': 'ankle',
    'subtalar_angle_r': 'ankle',
    'subtalar_angle_l': 'ankle',
    'knee_angle_r': 'knee',
    'knee_angle_l': 'knee',
    'soleus_r': 'soleus',
    'soleus_l': 'soleus',
    'tib_ant_r': 'tibialis_anterior',
    'med_gas_r': 'medial_gastrocnemius',
    'lat_gas_r': 'lateral_gastrocnemius',
}

# Initialize tracking variables
unfound_vars = set()
std_filled = 0
pelvis_sides_created = 0

# Cache to store created objects
file_object_cache = {}

# Mapping of exercise types to their corresponding models
EXERCISE_MODEL_MAP = {
    'run': Run,
    'walk': Walk,
    'jump': Jump,
    'squat': Squat,
    'land': Land,
    'lunge': Lunge,
}

def get_cache_key(model, kwargs):
    """Generate a unique cache key based on model and its kwargs."""
    # Convert all values to strings to handle non-hashable types
    sorted_items = sorted(kwargs.items())
    key = f"{model.__name__}-{json.dumps(sorted_items, default=str)}"
    return key

def get_or_create(model, **kwargs):
    """Get or create an instance of a model with caching."""
    key = get_cache_key(model, kwargs)
    if key in file_object_cache:
        return file_object_cache[key], False  # Assuming cached objects are not newly created
    instance, created = model.objects.get_or_create(**kwargs)
    file_object_cache[key] = instance
    return instance, created

def get_side_model(side, category):
    """Return the appropriate side model based on side and category."""
    side_suffix = 'LeftSide' if side == 'l' else 'RightSide'
    model_name = f"{category.capitalize()}{side_suffix}"
    return globals().get(model_name)  # Assumes models are imported in the global namespace

def load_data_into_db(file, type_exercise_unit, speed, data_type, mode, side, user, date=None):
    """Load data from a file into the database."""
    if date is None:
        date = datetime.now()

    logger.info(f"Loading File: {file}")
    
    try:
        data = pd.read_csv(file, sep='\t')
    except Exception as e:
        logger.error(f"Failed to read file {file}: {e}")
        return

    # Drop the first column if it's an unnamed index
    if data.columns[0].startswith('Unnamed'):
        data = data.drop(data.columns[0], axis=1)

    data.reset_index(drop=True, inplace=True)

    # Clear the cache for each file
    global file_object_cache
    file_object_cache = {}

    # Begin a database transaction for efficiency
    with transaction.atomic():
        # Iterate over each row except the last one
        for index, row in data.iterrows():
            # Get the exercise model based on type_exercise_unit
            exercise_model = EXERCISE_MODEL_MAP.get(type_exercise_unit)
            if not exercise_model:
                logger.warning(f"Unknown exercise type: {type_exercise_unit}")
                continue

            exercise_kwargs = {'date': date, 'user': user}
            exercise, created = get_or_create(exercise_model, **exercise_kwargs)

            # Prepare kwargs for ExerciseUnit
            exercise_unit_kwargs = {k: exercise if k == type_exercise_unit else None 
                                    for k in EXERCISE_MODEL_MAP.keys()}
            if type_exercise_unit in ['run', 'walk']:
                exercise_unit_kwargs['speed'] = speed
            exercise_unit, created = get_or_create(ExerciseUnit, **exercise_unit_kwargs)

            # Create or get GaitPhase
            gait_phase, created = get_or_create(GaitPhase, exercise_unit=exercise_unit, phase=index)

            # Create or get body part instances based on data_type
            body_parts = {}
            if data_type == 'ikAng':
                for part in ['pelvis', 'ankle', 'knee', 'hip']:
                    model = globals().get(part.capitalize())
                    if model:
                        body_part, created = get_or_create(model, gait_phase=gait_phase)
                        body_parts[part] = body_part
            elif data_type == 'musFor':
                for part in ['soleus', 'medial_gastrocnemius', 'lateral_gastrocnemius', 'tibialis_anterior']:
                    model = globals().get(part.capitalize())
                    if model:
                        body_part, created = get_or_create(model, gait_phase=gait_phase)
                        body_parts[part] = body_part

            # Handle side-specific models
            for category, instance in body_parts.items():
                if not instance:
                    continue
                side_model = get_side_model(side, category)
                if not side_model:
                    logger.warning(f"Side model not found for category: {category} and side: {side}")
                    continue

                side_instance, created = get_or_create(side_model, **{category: instance})
                if created and category == 'pelvis':
                    global pelvis_sides_created
                    pelvis_sides_created += 1

                # Set attributes based on data_type and mode
                for variable in data.columns:
                    if variable not in VAR_DICT:
                        unfound_vars.add(variable)
                        continue

                    var_category = VAR_DICT[variable]
                    value = row[variable]

                    if var_category != category:
                        continue


                    file_to_db_name = {
                        'ankle_angle_r': 'angle',
                        'ankle_angle_l': 'angle',
                        'subtalar_angle_r': 'subtalar_angle',
                        'subtalar_angle_l': 'subtalar_angle',
                        'knee_angle_r': 'angle',
                        'knee_angle_l': 'angle',
                        'hip_flexion_r': 'flexion',
                        'hip_flexion_l': 'flexion',
                        'hip_adduction_r': 'adduction',
                        'hip_adduction_l': 'adduction',
                        'hip_rotation_r': 'rotation',
                        'hip_rotation_l': 'rotation',
                    }

                    if data_type == 'ikAng':
                        suffix = '_angle_avg' if mode == 'Ave' else '_angle_std'
                        type_suffix = '_avg' if mode == 'Ave' else '_std'
                        if var_category in ['pelvis', 'hip', 'ankle', 'knee']:
                            if variable in file_to_db_name:
                                db_var = file_to_db_name[variable] + type_suffix
                            else:
                                db_var = variable.replace(f'{var_category}_', '').replace("_r", "").replace("_l", "") + suffix
                        else:
                            raise Exception(f"{var_category} not found in file_to_db_name")
                        # Make this error if not a db column on the side_instance
                        if not hasattr(side_instance, db_var):
                            raise Exception(f"Variable {db_var} not found in {side_instance}")
                        setattr(side_instance, db_var, value)

                    elif data_type == 'musFor':
                        suffix = '_avg' if mode == 'Ave' else '_std'
                        db_var = f'force{suffix}'
                        setattr(side_instance, db_var, value)

                # Save the side instance
                side_instance.save()

        if unfound_vars:
            logger.warning(f"Unfound variables: {unfound_vars}")



In [4]:
import re

def get_day_files(path):
    day_files = os.listdir(path)
    # Filter for following this regex day_*
    day_files = sorted([file for file in day_files if re.match(r'day_*', file)], key=lambda x: int(x.split('_')[1]))
    return day_files

In [5]:
user_data = {
    'weight': 75.2,
    'height': 180,
    'name': 'Test User 2 - Full Data Load'
}

#create a new User object
new_user = UserProfile.objects.create(**user_data)

#date = today minus 30 days
from datetime import datetime, timedelta

# build the full path for "./datasets/User_data/User1
path = os.path.join(os.getcwd(), "datasets/User_data/User1")


import os

#iterate over all day_{} folders inside the User_data/User1 folder
for day_folder in get_day_files(path):

    if not os.path.isdir(f"{path}/{day_folder}"):
        raise Exception(f"{day_folder} is not a directory")
    
    date = (datetime.now() - timedelta(days=-int(day_folder.split('_')[-1]))).date()
    print('processing day:', day_folder, "date:", date)

    # for each folder iterate over all files in the folder
    for day_number, file in enumerate(os.listdir(f"{path}/{day_folder}")):
        print('processing file:', file, "of day: ", day_folder)
        #check the file name to determine the type of data
        #split file name pattern by "_"
        file_name_parts = file.split("_")
        #iterate over filename parts to get the data type
        i = 1

        #get exercise unit type and it second part
        exercise_unit = file_name_parts[i]
        i += 1

        #get the speed from the next part if exercise is walk or run
        if exercise_unit in ['walk', 'run']:
            speed = file_name_parts[2]
            i+=1
        else:
            speed = None
        
        #get the type of data (ikAng, idTrq, musFor) and mode (Ave, Std) from this part which is like "ikAngAve"
        data_type = file_name_parts[i].replace('Ave', '').replace('Std', '')
        mode = file_name_parts[i].replace(data_type, '')
        i += 1

        #get the side from the last part
        side = file_name_parts[-1].replace('.txt', '')


        #if it is angle join data (ikAng) run the load angle data function
        if data_type == 'ikAng':
            load_data_into_db(f"{path}/{day_folder}/{file}", exercise_unit, speed, data_type, mode, side, new_user, date=date)  

        #if it is muscle force join data (musFor) run the load muscle force data function
        if data_type == 'musFor':
           load_data_into_db(f"{path}/{day_folder}/{file}", exercise_unit, speed,data_type, mode, side, new_user, date=date)

# print('Unfound variables:', unfound_vars)

INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_1/Subj04_run_81_ikAngAve_r.txt


processing day: day_1 date: 2024-10-19
processing file: Subj04_walk_09_idTrqStd_r.txt of day:  day_1
processing file: Subj04_walk_45_idTrqStd_l.txt of day:  day_1
processing file: Subj04_run_81_ikAngAve_r.txt of day:  day_1


INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_1/Subj04_jump_ikAngAve_l.txt


processing file: Subj04_jump_ikAngAve_l.txt of day:  day_1


INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_1/Subj04_walk_45_musForStd_r.txt


...

processing file: Subj04_jump_musForStd_l.txt of day:  day_23
processing file: Subj04_walk_09_idTrqAve_r.txt of day:  day_23
processing file: Subj04_walk_45_idTrqAve_l.txt of day:  day_23
processing file: Subj04_jump_ikAngStd_l.txt of day:  day_23


INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_23/Subj04_run_81_ikAngStd_r.txt


processing file: Subj04_run_81_ikAngStd_r.txt of day:  day_23


INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_23/Subj04_squat_musForAve_l.txt
INFO:__main__:Loading File: /Users/bekkblando/Documents/github/WearmAIoasis/wearmai/development/datasets/User_data/User1/day_23/Subj04_walk_45_musForAve_r.txt


processing file: Subj04_squat_musForAve_l.txt of day:  day_23
processing file: Subj04_walk_45_musForAve_r.txt of day:  day_23
